In [1]:
import pandas as pd
import numpy as np
import json
import os

# constants
DATAFRAME_PATH = 'dataframes'

### Create the playlists dataframe from raw JSON data

In [ ]:
# pre-load the pre-prepared all_tracks dataframe
all_tracks_df = pd.read_hdf(DATAFRAME_PATH + '/all_tracks.h5', 'tracks')
null_tracks = all_tracks_df[all_tracks_df.isnull().any(1)]['tracks']

playlist_track_info = []

def load_playlists(path):
    for file in os.listdir("raw_data"):
        if not file.endswith(".json"):
            continue
        with open("raw_data/"+file) as f:
            js_slice = json.load(f)
            for playlist in js_slice['playlists']:
                for track in playlist['tracks']:
                    if track['track_uri'] not in null_tracks:
                        playlist_track_info.append([track['track_uri'], playlist['pid'], track['pos']])
                playlist.pop('tracks')
                yield playlist
        print(f"Added playlists from {file}")

main_playlist_df = pd.DataFrame(load_playlists(DATAFRAME_PATH + '/playlists.h5'))
playlist_track_info_df = pd.DataFrame(playlist_track_info, columns=['track_uri', 'pid', 'pos'])
playlist_track_info_df.set_index('pid')

print('Storing h5 files...')
main_playlist_df.to_hdf(DATAFRAME_PATH + '/playlists.h5', 'playlists')
playlist_track_info_df.to_hdf(DATAFRAME_PATH + '/playlist_track_info.h5', 'track_info')
print('Stored files as h5')

### Read the playlists, tracks and track_info dataframes

In [2]:
playlists_df = pd.read_hdf(DATAFRAME_PATH + '/playlists.h5', 'playlists')
tracks_df = pd.read_hdf(DATAFRAME_PATH + '/clean_tracks.h5', 'tracks')
track_info_df = pd.read_hdf(DATAFRAME_PATH + '/playlist_track_info.h5', 'track_info')

## Building the train and test sets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(playlists_df, test_size=0.2)

In [4]:
# Modifying the test set
X_train_track_info = track_info_df[track_info_df['pid'].isin(X_train['pid'].values)]
X_test_track_info = track_info_df[track_info_df['pid'].isin(X_test['pid'].values)]

# Keep only tracks that are in the training set in the test set 
X_train_track_uris = pd.unique(X_train_track_info['track_uri'])
X_test_track_info_ground_truth = X_test_track_info[X_test_track_info['track_uri'].isin(X_train_track_uris)]

In [5]:
# some assertions on the ground truth set
assert X_test_track_info_ground_truth[X_test_track_info_ground_truth['track_uri'].isin(X_train_track_info['track_uri']) == False].empty

In [ ]:
# This cell can take long to run ~10mins
# if .h5 files are available, use those

challenge_set_list = []
new_test_set = []

def frac_to_sample(playlist_tracks):
    if playlist_tracks.size >= 2:
        return 0.5
    else:
        return None
        
def build_challenge_set():
    for pid in X_test['pid']:
        playlist_tracks = X_test_track_info_ground_truth[X_test_track_info_ground_truth['pid'] == pid]
        frac = frac_to_sample(playlist_tracks)
        if frac is not None and frac > 0:
            random_sample_df = playlist_tracks.sample(frac = frac, random_state=1)
            num_sample = random_sample_df.size
            new_test_set.append(X_test[X_test['pid'] == pid].values.flatten().tolist() + [num_sample, playlist_tracks.size-num_sample])
            for index, track in random_sample_df.iterrows():
                challenge_set_list.append([track['track_uri'], track['pid'], track['pos']])
    
build_challenge_set()

In [7]:
X_challenge = pd.DataFrame(new_test_set, columns=list(X_test.columns) + ['num_of_seeds', 'num_withheld'])
X_challenge_track_info = pd.DataFrame(challenge_set_list, columns=list(X_test_track_info_ground_truth.columns))

# Write challenge set as hdf
X_challenge.to_hdf(DATAFRAME_PATH + '/challenge_playlists.h5', 'playlists')
X_challenge_track_info.to_hdf(DATAFRAME_PATH + '/challenge_playlist_track_info.h5', 'track_info')

C:\Users\Siddharth\anaconda3\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['name', 'collaborative', 'description'], dtype='object')]

  pytables.to_hdf(


In [8]:
# Load challenge dataframes
X_challenge_set = pd.read_hdf(DATAFRAME_PATH + '/challenge_playlists.h5', 'playlists')
X_challenge_set_track_info = pd.read_hdf(DATAFRAME_PATH + '/challenge_playlist_track_info.h5', 'track_info')

In [9]:
# some assertions on the challenge set
assert X_challenge_set[X_challenge_set_track_info['pid'].isin(X_challenge_set['pid']) == False].empty
assert X_challenge_set[X_challenge_set_track_info['track_uri'].isin(X_train_track_info['track_uri']) == False].empty
assert X_challenge_set_track_info.size < X_test_track_info_ground_truth.size

<ipython-input-9-532c307d9ef3>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  assert X_challenge_set[X_challenge_set_track_info['pid'].isin(X_challenge_set['pid']) == False].empty
<ipython-input-9-532c307d9ef3>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  assert X_challenge_set[X_challenge_set_track_info['track_uri'].isin(X_train_track_info['track_uri']) == False].empty


## Building the model

In [10]:
import sklearn
import sklearn.neighbors

In [11]:
# hyperparameters
num_neighbours = 10

In [12]:
knn_clf = sklearn.neighbors.NearestNeighbors(n_neighbors=num_neighbours)

In [18]:
selected_features = ['danceability', 'energy', 'key', 'loudness',
                     'speechiness', 'acousticness', 'instrumentalness',
                     'liveness', 'valence', 'tempo']

# knn_features = X_train_track_info[selected_features]

In [16]:
print(len(X_train_track_uris))
print(len(tracks_df))
# Missing some 28 tracks? TODO
X_train_features = tracks_df[tracks_df['tracks'].isin(X_train_track_uris)]

994072
1119279


In [17]:
knn_clf.fit(X_train_features[selected_features])

NearestNeighbors(n_neighbors=10)

In [19]:
#knn_clf.n_samples_fit_
#distances, indices = nbrs.kneighbors(X_)
#X_test
print(len(X_test_track_info_ground_truth))
X_test_track_uris = pd.unique(X_test_track_info_ground_truth['track_uri'])
len(X_test_track_uris)

X_test_features = tracks_df[tracks_df['tracks'].isin(X_test_track_uris)]

3207894


In [20]:
distances, indices = knn_clf.kneighbors(X_test_features[selected_features])

In [21]:
#X_test_features.loc[0]
#X_train_features.iloc[indices[0]].index
test_idx = [0, 292891, 176330, 785063, 347886]
X_train_features.reset_index().iloc[indices[1]]

,index,tracks,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1,1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.774,0.838,5.0,-3.914,0.0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0
980274,1102206,spotify:track:6fEBlVQSTYx6Ej0QhXPyb0,0.742,0.737,5.0,-3.992,1.0,0.0558,0.136000,0.000000,0.0908,0.896,142.990,audio_features,6fEBlVQSTYx6Ej0QhXPyb0,spotify:track:6fEBlVQSTYx6Ej0QhXPyb0,https://api.spotify.com/v1/tracks/6fEBlVQSTYx6...,https://api.spotify.com/v1/audio-analysis/6fEB...,185147.0,4.0
42395,43054,spotify:track:65QVpgBIutEZlpDCi6AH2F,0.640,0.941,5.0,-4.008,0.0,0.0893,0.012300,0.000000,0.3050,0.761,142.905,audio_features,65QVpgBIutEZlpDCi6AH2F,spotify:track:65QVpgBIutEZlpDCi6AH2F,https://api.spotify.com/v1/tracks/65QVpgBIutEZ...,https://api.spotify.com/v1/audio-analysis/65QV...,164880.0,4.0
368059,387126,spotify:track:5hZ2i7qmmDCzlDRtxoE5oh,0.722,0.891,5.0,-4.067,1.0,0.0529,0.204000,0.001240,0.1050,0.928,142.917,audio_features,5hZ2i7qmmDCzlDRtxoE5oh,spotify:track:5hZ2i7qmmDCzlDRtxoE5oh,https://api.spotify.com/v1/tracks/5hZ2i7qmmDCz...,https://api.spotify.com/v1/audio-analysis/5hZ2...,244605.0,4.0
466982,495793,spotify:track:7k6ZQrCSD98mFIDRatLyCj,0.538,0.972,5.0,-3.680,0.0,0.0398,0.000735,0.000000,0.1470,0.807,143.031,audio_features,7k6ZQrCSD98mFIDRatLyCj,spotify:track:7k6ZQrCSD98mFIDRatLyCj,https://api.spotify.com/v1/tracks/7k6ZQrCSD98m...,https://api.spotify.com/v1/audio-analysis/7k6Z...,208453.0,4.0
263286,273368,spotify:track:4c6gceS55MEjEXz4FI0yet,0.665,0.795,5.0,-4.222,1.0,0.0868,0.012500,0.000016,0.1470,0.705,142.895,audio_features,4c6gceS55MEjEXz4FI0yet,spotify:track:4c6gceS55MEjEXz4FI0yet,https://api.spotify.com/v1/tracks/4c6gceS55MEj...,https://api.spotify.com/v1/audio-analysis/4c6g...,130293.0,4.0
410325,432890,spotify:track:0KP6ZRIx0ncKnzlm44S1s0,0.633,0.653,5.0,-3.966,1.0,0.0419,0.254000,0.000000,0.0731,0.722,143.133,audio_features,0KP6ZRIx0ncKnzlm44S1s0,spotify:track:0KP6ZRIx0ncKnzlm44S1s0,https://api.spotify.com/v1/tracks/0KP6ZRIx0ncK...,https://api.spotify.com/v1/audio-analysis/0KP6...,229893.0,4.0
714964,779830,spotify:track:4FjtXTWHozZLfxfMJLAmyA,0.591,0.733,5.0,-3.754,1.0,0.0286,0.271000,0.000343,0.1290,0.692,142.978,audio_features,4FjtXTWHozZLfxfMJLAmyA,spotify:track:4FjtXTWHozZLfxfMJLAmyA,https://api.spotify.com/v1/tracks/4FjtXTWHozZL...,https://api.spotify.com/v1/audio-analysis/4Fjt...,198413.0,4.0
884002,984732,spotify:track:0LFjHJnsZd1B6KPnvTmIVy,0.536,0.785,5.0,-4.080,1.0,0.0481,0.205000,0.000736,0.1500,0.649,143.140,audio_features,0LFjHJnsZd1B6KPnvTmIVy,spotify:track:0LFjHJnsZd1B6KPnvTmIVy,https://api.spotify.com/v1/tracks/0LFjHJnsZd1B...,https://api.spotify.com/v1/audio-analysis/0LFj...,270173.0,4.0
71816,73295,spotify:track:7hfLxeKcjVkaw2tgw5IH9Q,0.670,0.840,5.0,-3.913,0.0,0.0513,0.200000,0.000000,0.4210,0.553,142.980,audio_features,7hfLxeKcjVkaw2tgw5IH9Q,spotify:track:7hfLxeKcjVkaw2tgw5IH9Q,https://api.spotify.com/v1/tracks/7hfLxeKcjVka...,https://api.spotify.com/v1/audio-analysis/7hfL...,232413.0,4.0


In [22]:
X_challenge_set[X_challenge_set['pid'] == 106052]
#X_test_track_info_ground_truth[X_test_track_info_ground_truth['pid'] == ]

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms,num_artists,description,num_of_seeds,num_withheld


In [23]:
X_test_features.iloc[:2]

,tracks,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,4.0,-7.105,0.0,0.121,0.0311,0.00697,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.774,0.838,5.0,-3.914,0.0,0.114,0.0249,0.02500,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0


In [56]:
tracks_df = tracks_df.rename(columns={'tracks':'track_uri'})
X_challenge_track_info_features = X_challenge_track_info.merge(tracks_df, how='outer', on='track_uri').dropna()
X_challenge_track_info_features.head()

,track_uri,pid,pos,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,spotify:track:35WhawODuOs0kaHxwmPA9D,117336.0,62.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266,...,0.119,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0
1,spotify:track:35WhawODuOs0kaHxwmPA9D,148868.0,53.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266,...,0.119,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0
2,spotify:track:35WhawODuOs0kaHxwmPA9D,230166.0,102.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266,...,0.119,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0
3,spotify:track:35WhawODuOs0kaHxwmPA9D,156656.0,45.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266,...,0.119,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0
4,spotify:track:35WhawODuOs0kaHxwmPA9D,102613.0,58.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266,...,0.119,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0


In [110]:
# function to read playlist from challenge set


list_mean = []

for _, playlist in X_challenge_set.iterrows():
    tracks_with_features_in_playlist = X_challenge_track_info_features[X_challenge_track_info_features['pid'] == playlist['pid']]
    mean = tracks_with_features_in_playlist[selected_features].mean()
    mean['pid'] = playlist['pid']
#     print(mean)
    list_mean.append(mean)
    # store for all pids
    
playlists_mean_features = pd.DataFrame(data=list_mean , columns=[*selected_features, 'pid'])

In [64]:
distances, indices = knn_clf.kneighbors(tracks_with_features_in_playlist[selected_features])

In [ ]:
print(distances)
print(indices)
X_train_features.reset_index().iloc[indices[0]]

In [ ]:
distances, indices = knn_clf.kneighbors(tracks_with_features_in_playlist[selected_features].mean())

In [82]:
tracks_with_features_in_playlist[selected_features].mean()

danceability          0.606481
energy                0.535135
key                   4.615385
loudness             -7.813154
speechiness           0.115812
acousticness          0.222630
instrumentalness      0.006320
liveness              0.176900
valence               0.352013
tempo               118.324942
dtype: float64

In [79]:
tracks_with_features_in_playlist

,track_uri,pid,pos,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,spotify:track:35WhawODuOs0kaHxwmPA9D,117336.0,62.0,0.518,0.492,11.0,-11.085,0.0,0.0345,0.266000,...,0.1190,0.0952,131.979,audio_features,35WhawODuOs0kaHxwmPA9D,spotify:track:35WhawODuOs0kaHxwmPA9D,https://api.spotify.com/v1/tracks/35WhawODuOs0...,https://api.spotify.com/v1/audio-analysis/35Wh...,257573.0,4.0
5,spotify:track:60kqoWSXSbiDJaDxgi8GG1,117336.0,40.0,0.550,0.661,1.0,-7.363,1.0,0.0611,0.017800,...,0.1500,0.1420,162.034,audio_features,60kqoWSXSbiDJaDxgi8GG1,spotify:track:60kqoWSXSbiDJaDxgi8GG1,https://api.spotify.com/v1/tracks/60kqoWSXSbiD...,https://api.spotify.com/v1/audio-analysis/60kq...,313907.0,4.0
10,spotify:track:1eQBEelI2NCy7AUTerX0KS,117336.0,39.0,0.592,0.403,0.0,-8.768,0.0,0.4140,0.620000,...,0.5400,0.3910,105.169,audio_features,1eQBEelI2NCy7AUTerX0KS,spotify:track:1eQBEelI2NCy7AUTerX0KS,https://api.spotify.com/v1/tracks/1eQBEelI2NCy...,https://api.spotify.com/v1/audio-analysis/1eQB...,320680.0,4.0
23,spotify:track:67ispalOAo3jnZWYSTTfZk,117336.0,69.0,0.408,0.974,2.0,-4.139,1.0,0.1260,0.000013,...,0.5260,0.1730,148.298,audio_features,67ispalOAo3jnZWYSTTfZk,spotify:track:67ispalOAo3jnZWYSTTfZk,https://api.spotify.com/v1/tracks/67ispalOAo3j...,https://api.spotify.com/v1/audio-analysis/67is...,207002.0,4.0
27,spotify:track:2ekdnv2xVIyAdG3ySoDDeY,117336.0,105.0,0.344,0.537,8.0,-8.215,0.0,0.0790,0.072700,...,0.2250,0.4530,82.878,audio_features,2ekdnv2xVIyAdG3ySoDDeY,spotify:track:2ekdnv2xVIyAdG3ySoDDeY,https://api.spotify.com/v1/tracks/2ekdnv2xVIyA...,https://api.spotify.com/v1/audio-analysis/2ekd...,212880.0,5.0
29,spotify:track:1VWRcdvlTztNh2FHwKujUG,117336.0,38.0,0.517,0.715,5.0,-9.345,0.0,0.2820,0.137000,...,0.1490,0.5960,130.069,audio_features,1VWRcdvlTztNh2FHwKujUG,spotify:track:1VWRcdvlTztNh2FHwKujUG,https://api.spotify.com/v1/tracks/1VWRcdvlTztN...,https://api.spotify.com/v1/audio-analysis/1VWR...,210673.0,4.0
31,spotify:track:6PGoSes0D9eUDeeAafB2As,117336.0,51.0,0.800,0.585,10.0,-7.343,1.0,0.0924,0.264000,...,0.1530,0.7790,126.058,audio_features,6PGoSes0D9eUDeeAafB2As,spotify:track:6PGoSes0D9eUDeeAafB2As,https://api.spotify.com/v1/tracks/6PGoSes0D9eU...,https://api.spotify.com/v1/audio-analysis/6PGo...,213400.0,4.0
47,spotify:track:5tFep7dXGd7vEJ668wTPux,117336.0,88.0,0.850,0.405,7.0,-7.509,0.0,0.2250,0.119000,...,0.0707,0.3440,139.976,audio_features,5tFep7dXGd7vEJ668wTPux,spotify:track:5tFep7dXGd7vEJ668wTPux,https://api.spotify.com/v1/tracks/5tFep7dXGd7v...,https://api.spotify.com/v1/audio-analysis/5tFe...,190312.0,4.0
61,spotify:track:6Xs7OKt6i0Ddq8B4M5Dgvr,117336.0,65.0,0.339,0.707,0.0,-5.121,0.0,0.0428,0.011300,...,0.3370,0.3590,173.993,audio_features,6Xs7OKt6i0Ddq8B4M5Dgvr,spotify:track:6Xs7OKt6i0Ddq8B4M5Dgvr,https://api.spotify.com/v1/tracks/6Xs7OKt6i0Dd...,https://api.spotify.com/v1/audio-analysis/6Xs7...,183808.0,4.0
67,spotify:track:0PDUDa38GO8lMxLCRc4lL1,117336.0,54.0,0.584,0.700,11.0,-4.275,1.0,0.0456,0.117000,...,0.0939,0.4380,124.944,audio_features,0PDUDa38GO8lMxLCRc4lL1,spotify:track:0PDUDa38GO8lMxLCRc4lL1,https://api.spotify.com/v1/tracks/0PDUDa38GO8l...,https://api.spotify.com/v1/audio-analysis/0PDU...,202747.0,4.0


In [112]:
playlists_mean_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      2015 non-null   float64
 1   energy            2015 non-null   float64
 2   key               2015 non-null   float64
 3   loudness          2015 non-null   float64
 4   speechiness       2015 non-null   float64
 5   acousticness      2015 non-null   float64
 6   instrumentalness  2015 non-null   float64
 7   liveness          2015 non-null   float64
 8   valence           2015 non-null   float64
 9   tempo             2015 non-null   float64
 10  pid               2016 non-null   float64
dtypes: float64(11)
memory usage: 173.4 KB
